In [1]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("test").getOrCreate()
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1632053667673_0012,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%sh
sudo python3 -m pip install boto3
sudo python3 -m pip install mleap


You are using pip version 19.0.2, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 21.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'mleap'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'mleap'



In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
11,application_1632053667673_0012,pyspark,idle,Link,Link,None,✔


In [ ]:
%%sh
wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar

In [5]:
from pyspark.sql.functions import to_timestamp,hour,minute,when,col,current_timestamp,date_format,lit,unix_timestamp,expr,abs,to_date,rank,datediff
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder,StringIndexer,VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator,TrainValidationSplit
from pyspark.ml.regression import LinearRegression,RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import os

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import sagemaker_pyspark
import botocore.session

session = botocore.session.get_session()
# credentials = session.get_credentials()

conf = (SparkConf().set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars())))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'botocore'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'botocore'



In [7]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.config(conf = conf).appName("test").getOrCreate()
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'conf' is not defined
Traceback (most recent call last):
NameError: name 'conf' is not defined



In [8]:
rawdata_s3 = 's3a://solarpowerbackend/rawdata/'
processed_s3 = 's3a://solarpowerbackend/processed/'
model_s3 = 's3a://solarpowerbackend/model/'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
raw_df = spark.read.parquet(f'{processed_s3}rul_dataset')
raw_df = raw_df.withColumn('TIME',date_format(col('DATE_TIME'),'HH:mm:ss'))
raw_df = raw_df.filter(col('TIME').between('06:00:00','18:15:00'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
raw_df = raw_df.withColumn('DATE',to_date(col('DATE_TIME')))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
only_fault_df = raw_df.select('DATE_TIME','FAULT_FLAG','SOURCE_KEY','DATE').filter(col('Fault_Flag')==1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
display(only_fault_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'display' is not defined
Traceback (most recent call last):
NameError: name 'display' is not defined



In [13]:
# rank_window = Window.partitionBy(only_fault_df['SOURCE_KEY'],only_fault_df['DATE']).orderBy(only_fault_df['DATE_TIME'])
# fault_rank_on_day = rank().over(rank_window)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
fault_rank_df = only_fault_df.select(col('SOURCE_KEY').alias('FAULT_SOURCE_KEY'),col('DATE_TIME').alias('FAULT_DATE_TIME'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
rul_oncondition = (fault_rank_df['FAULT_DATE_TIME']>=raw_df['DATE_TIME']) & (fault_rank_df['FAULT_SOURCE_KEY'] == raw_df['SOURCE_KEY'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
rul_df = fault_rank_df.join(raw_df,on = rul_oncondition,how = 'inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
display(rul_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'display' is not defined
Traceback (most recent call last):
NameError: name 'display' is not defined



In [18]:
rul_df = rul_df.withColumn('RUL',(col('FAULT_DATE_TIME').cast('long') - col('DATE_TIME').cast('long'))/60 - datediff(col('FAULT_DATE_TIME'),col('DATE_TIME'))*47*15)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
display(rul_df)

In [19]:
rul_df = rul_df.groupby('SOURCE_KEY','DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','PLANT_ID','AC_POWER','DC_POWER','DAILY_YIELD','TOTAL_YIELD').min('RUL')
rul_df = rul_df.withColumnRenamed('min(RUL)','RUL')
# rul_df = rul_df.filter(col('RUL')>=1440)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
rul_df = rul_df.withColumn('AC_POWER',col('AC_POWER') - col('AC_POWER')/(col('RUL')+1)*2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
rul_df = rul_df.withColumn('ACDC',col('AC_POWER')/(col('DC_POWER')+1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
display(rul_df.sort(['SOURCE_KEY','DATE_TIME']).filter(col('RUL')<100))

In [22]:

str_ind = StringIndexer().setInputCol('SOURCE_KEY').setOutputCol('SOURCE_KEY_NUM')
ohe = OneHotEncoder().setInputCol('SOURCE_KEY_NUM').setOutputCol('encoded_Source_Key')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
index_stages = [str_ind,ohe]
ohe_pipeline = Pipeline(stages = index_stages)
rul_indexed_df = ohe_pipeline.fit(rul_df).transform(rul_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
independent_features = ['AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','AC_POWER','ACDC','DC_POWER','DAILY_YIELD','encoded_Source_Key']
model_input_vector = VectorAssembler().setInputCols(independent_features).setOutputCol('features')

rul_indexed_df = rul_indexed_df.withColumnRenamed('RUL','label')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
rul_indexed_train_df,rul_indexed_test_df = rul_indexed_df.randomSplit([0.8,0.2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
lr_model = LinearRegression().setFeaturesCol('features').setLabelCol('label')
lr_model.setPredictionCol('LR_prediction')
rf_model = RandomForestRegressor().setFeaturesCol('features').setLabelCol('label')
rf_model.setPredictionCol('RF_prediction')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForestRegressor_4404f8298527

In [27]:
model_training_stages  = [model_input_vector]
rul_pipeline = Pipeline(stages=model_training_stages)
rul_trained_df = rul_pipeline.fit(rul_indexed_train_df).transform(rul_indexed_train_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
rfparamGrid = (ParamGridBuilder()
             .addGrid(rf_model.maxDepth, [2, 5, 10, 20, 30])
#                .addGrid(rf_model.maxDepth, [2, 5, 10])
             .addGrid(rf_model.maxBins, [10, 20, 40, 80, 100])
#                .addGrid(rf_model.maxBins, [5, 10, 20])
             .addGrid(rf_model.numTrees, [5, 20, 50, 100, 500])
#                .addGrid(rf_model.numTrees, [5, 20, 50])
             .build())
rfevaluator = RegressionEvaluator(predictionCol="RF_prediction", labelCol="label", metricName="rmse")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
rfcv = CrossValidator(estimator = rf_model,
                      estimatorParamMaps = rfparamGrid,
                      evaluator = rfevaluator,
                      numFolds = 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
rfcvModel = rfcv.fit(rul_trained_df)
print(rfcvModel)

# Use test set here so we can measure the accuracy of our model on new data
rfpredictions = rfcvModel.transform(rul_trained_df)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
print('RMSE:', rfevaluator.evaluate(rfpredictions))

In [30]:


rul_lr_model = lr_model.fit(rul_trained_df)
rul_trained_df = rul_lr_model.transform(rul_trained_df)
print('Linear Regression Error: ' ,str(rul_lr_model.summary.meanAbsoluteError))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear Regression Error:  1487.873825021676

In [31]:
rul_trained_df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(SOURCE_KEY='1BY6WEcLGh8j5v7', DATE_TIME=datetime.datetime(2020, 5, 15, 17, 15), AMBIENT_TEMPERATURE=33.30025473333334, MODULE_TEMPERATURE=35.852207400000005, IRRADIATION=0.17389863733333336, PLANT_ID=4135001, AC_POWER=196.83252700270026, DC_POWER=2011.375, DAILY_YIELD=5687.75, TOTAL_YIELD=6265246.75, label=1110.0, ACDC=0.09781105758255805, SOURCE_KEY_NUM=4.0, encoded_Source_Key=SparseVector(21, {4: 1.0}), features=SparseVector(28, {0: 33.3003, 1: 35.8522, 2: 0.1739, 3: 196.8325, 4: 0.0978, 5: 2011.375, 6: 5687.75, 11: 1.0}), LR_prediction=2800.6544567864985)

In [ ]:
rul_rf_model.write().overwrite().save(f'{model_s3}RF_RUL_Model')

In [ ]:
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer
SimpleSparkSerializer().serializeToBundle(
    rul_rf_model, "jar:file:/tmp/model.zip", rul_trained_df
)

In [ ]:
rul_rf_model.serializeToBundle(f"jar:{model_s3}rul_rf_model.zip",rul_trained_df )

In [32]:
rul_rf_model = rf_model.fit(rul_trained_df)
rul_trained_df = rul_rf_model.transform(rul_trained_df)
# print('Randomforest Regression Error: ' ,str(rul_rf_model.stages[-1].summary.meanAbsoluteError))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
SimpleSparkSerializer().serializeToBundle(
    model, "jar:file:/tmp/model.zip", transformed_validation_df
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'SimpleSparkSerializer' is not defined
Traceback (most recent call last):
NameError: name 'SimpleSparkSerializer' is not defined



In [ ]:
rul_rf_model.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
rul_trained_df.select('SOURCE_KEY').distinct().collect()

In [ ]:
display(rul_trained_df.filter(col('SOURCE_KEY')=='bvBOhCH3iADSZry').select('SOURCE_KEY','DATE_TIME','RF_Prediction','LR_Prediction','label'))

In [ ]:
rul_trained_df = rul_trained_df.withColumn('acdc',col('DC_POWER')/col('AC_POWER'))

In [ ]:
display(rul_trained_df.filter(col('SOURCE_KEY')=='1BY6WEcLGh8j5v7').select('DATE_TIME',*(independent_features[:-1]),'label'))